In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
train_df = pd.read_csv('WineData.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.shape
train_df.describe()
train_df.columns
train_df.index

In [ ]:
sns.pairplot(train_df)

In [ ]:
# Check May 28, 7:38 AM

In [ ]:
# Check May 28, 7:42 AM